In [1047]:
from sympy import symbols, Eq, simplify, And, Or, Not
from sympy.logic import simplify_logic
from sympy.logic.boolalg import to_dnf
# from sympy.logic.boolalg import factorbool

import sympy.logic.boolalg as b
print(dir(b))



['ANFform', 'Add', 'And', 'Any', 'Application', 'Basic', 'Boolean', 'BooleanAtom', 'BooleanFalse', 'BooleanFunction', 'BooleanGates', 'BooleanKind', 'BooleanTrue', 'Derivative', 'Equivalent', 'Exclusive', 'ITE', 'Implies', 'Iterable', 'LatticeOp', 'Mapping', 'Nand', 'Nor', 'Not', 'Number', 'NumberKind', 'Or', 'POSform', 'S', 'SOPform', 'Singleton', 'TYPE_CHECKING', 'Tuple', 'Xnor', 'Xor', '__builtins__', '__cached__', '__doc__', '__file__', '__loader__', '__name__', '__package__', '__spec__', '_apply_patternbased_simplification', '_apply_patternbased_threeterm_simplification', '_apply_patternbased_twoterm_simplification', '_check_pair', '_convert_to_varsANF', '_convert_to_varsPOS', '_convert_to_varsSOP', '_distribute', '_find_predicates', '_finger', '_get_even_parity_terms', '_get_odd_parity_terms', '_get_truthtable', '_input_to_binlist', '_is_form', '_rem_redundancy', '_simplified_pairs', '_simplify_patterns_and', '_simplify_patterns_and3', '_simplify_patterns_or', '_simplify_patterns

In [1048]:
# sympy example use part 0

X = [symbols("X[0]"),symbols("X[1]"),symbols("X[2]")]

A,B,C,D,E,F,G = symbols("A B C D E F G")

expr = E | (A & E) | (B & E) | (C & True) | (D & E)

# expr = expr.subs({A : 3})

print(simplify(expr))  # still returns (A & B) | (A & ~B)

C | E


In [1049]:
# sympy example use part 1

# v = ['(', '(', 'x[0]', '==', 1, ')', 'AND', '(', '(', '(', 'x[1]', '==', 0, ')', 'AND', '(', 'l[1][0]', ')', ')', 'OR', '(', '(', 'x[2]', '==', 0, ')', 'AND', '(', 'l[1][0]', ')', ')', 'OR', '(', '(', 'x[3]', '==', 0, ')', 'AND', '(', 'l[1][0]', ')', ')', ')', ')']

In [1050]:
# sympy example use part 2

# l = [[[] for t in range(10)] for w in range(10)]

# for t in range(10):
#   for w in range(10):
#     l[w][t] = symbols(f"l[{w}][{t}]")

# x = [[] for i in range(4)]

# for i in range(4):
#   x[i] = symbols(f"x[{i}]")

# v_prime = []

# i = 0
# while i < len(v):
#   if v[i] == "AND": v_prime.append("&"); i+=1 #; print(v_prime); print(1)
#   elif v[i] == "OR": v_prime.append("|"); i+=1 #; print(v_prime); print(2)
#   elif v[i][0] == "x":
#     v_prime.append(f"Eq({v[i]},{v[i+2]})")
#     i+=3 #; print(v_prime); print(3)
#   else:
#     v_prime.append(v[i]); i+=1 #; print(v_prime); print(4)

# # print(v_prime)
# print(" ".join(v_prime))
# print()

# expr = eval(" ".join(v_prime))

# expr2 = expr.subs({l[1][0] : Eq(x[0],0)})

# simp = simplify(simplify_logic(expr2))

# print(expr)
# print()
# print(expr2)
# print()
# print(simp)
# print()
# print(str(expr))

In [1051]:
def clean_vector_sympy(v, t_max, w_max, b_max):
    # ---------------- Maybe put all this OUTSIDE somewhere ----------------
    L = [[[] for t in range(t_max)] for w in range(w_max)]
    X = [[] for b in range(b_max)]

    for t in range(t_max):
        for w in range(w_max):
            L[w][t] = symbols(f"L[{w}][{t}]")
    
    for b in range(b_max):
        X[b] = symbols(f"X[{b}]")
    # ---------------- ---------------- ---------------- ----------------
    

    # Making 0 into False and 1 into True
    # Replacing x with X, so we use X[] and not x[] by accident
    # Replacing l with L, so we use L[][] and not l[][] by accident
    v2 = ['False' if i == '0' else
          'True'  if i == '1' else
          i.translate(str.maketrans({"x": "X", "l": "L"}))
          for i in v]

    # Putting all together in a string
    v_as_string = " ".join(v2)

    
    # v_as_string = v_as_string.replace("x", "X")
    # v_as_string = v_as_string.replace("l", "L")

    # print(v_as_string)
    expr = eval(v_as_string)
    # expr = expr.subs({l[1][0] : Eq(x[0],0)})

    simplified_expr = simplify(simplify_logic(expr))
    # simplified_expr = simplify(to_dnf(simplify(simplify_logic(expr)), simplify=True,force=True))

    return str(simplified_expr)

In [1052]:
# Global Variables for program

v = [] # vector storing the ending expression

set_of_ks = {} # set that stores values like K(b,w,t,bv) and k(b,s_d,w,t)

v_of_vocab_sets = []

In [1053]:
def iterate_B(B): # iterates through Bs terms, 
    # places values in set_of_ks,
    # TBD (use b_vocab) -> changes B to eliminate redundancy,
    # TBD () -> removes book b from B if no word to be learned (B[b] is an empty list)

    
    for b in range(len(B)):
        b_vocab = set()
        for s_d in range(len(B[b])):
            
            b_vocab.update(B[b][s_d])
            # Add all words in s_d to vocabulary

            if len(B[b][s_d]) == 1:

                w = B[b][s_d][0]
                base_K = f"K({b},{w},0,[])"
                set_of_ks[base_K] = ['1']
                # if s_d of book b has 1 word (w), 
                # we register that we can learn w by reading book b
                # -------- 1 --------

                w = B[b][s_d][0]
                base_K = f"K_reread({b},{w},0)"
                set_of_ks[base_K] = ['1']
                # if s_d of book b has 1 word (w), 
                # we register that we can learn w by reading book b



        
        # Add set of book vocab to the global list
        v_of_vocab_sets.append(b_vocab)
                

In [1054]:
def print_vector(vec):
    for x in vec:
        print(x, end=" ")
    print()  # final newline


In [1055]:
def vector_x_eq_int(b, t):
    # x_eq_int = ["(",f"Eq(x[{b}],{t})",")"]
    x_eq_int = [f"Eq(x[{b}],{t})"]
    return x_eq_int

def andify(vector_of_ands): # vector_of_ands is a vector of vectors
    v = []
    and_symbol_string = "&"
    
    for i in range(len(vector_of_ands)):
        v.append("(")
        if vector_of_ands[i] == []:
            # if empty its the same as AND 1
            v.append('1')
        else:
            v.extend(vector_of_ands[i])
        v.append(")")
        if i != len(vector_of_ands)-1: 
            # if not the last, append an "AND" after
            v.append(and_symbol_string)
    return v

# def andify(vector_of_ands): # vector_of_ands is a vector of vectors
#     v = []
#     and_symbol_string = ","
    
#     v.append("And")
#     for i in range(len(vector_of_ands)):
#         v.append("(")
#         if vector_of_ands[i] == []:
#             # if empty its the same as AND 1
#             v.append('1')
#         else:
#             v.extend(vector_of_ands[i])
#         v.append(")")
#         if i != len(vector_of_ands)-1: 
#             # if not the last, append an "AND" after
#             v.append(and_symbol_string)
#     return v

def orify(vector_of_ors): # vector_of_ors is a vector of vectors
    v = []
    or_symbol_string = "|"
    
    for i in range(len(vector_of_ors)):
        v.append("(")
        if vector_of_ors[i] == []: 
            # if empty its the same as OR 0
            v.append('0')
        else:
            v.extend(vector_of_ors[i])
        v.append(")")
        if i != len(vector_of_ors)-1: 
            # if not the last, append an "OR" after
            v.append(or_symbol_string)
    return v

# def orify(vector_of_ors): # vector_of_ors is a vector of vectors
#     v = []
#     or_symbol_string = ","
    
#     v.append("Or")
#     for i in range(len(vector_of_ors)):
#         v.append("(")
#         if vector_of_ors[i] == []: 
#             # if empty its the same as OR 0
#             v.append('0')
#         else:
#             v.extend(vector_of_ors[i])
#         v.append(")")
#         if i != len(vector_of_ors)-1: 
#             # if not the last, append an "OR" after
#             v.append(or_symbol_string)
#     return v


In [1056]:
andify([["E"]])

['(', 'E', ')']

In [1057]:
def clean_constants(v_prime): # by ChatGPT # removes 0 OR or OR 0 and 1 AND or AND 1
    v = v_prime.copy()
    i = 0
    while i < len(v) - 2:
        a, op, b = v[i], v[i+1], v[i+2]

        # Case: 0 OR X → X
        if a == '0' and op == "OR":
            del v[i:i+2]   # remove 0, "OR"
            continue

        # Case: X OR 0 → X
        if op == "OR" and b == '0':
            del v[i+1:i+3]  # remove "OR", 0
            continue

        # Case: 1 AND X → X
        if a == '1' and op == "AND":
            del v[i:i+2]   # remove 1, "AND"
            continue

        # Case: X AND 1 → X
        if op == "AND" and b == '1':
            del v[i+1:i+3]  # remove "AND", 1
            continue

        i += 1
    return v

def collapse_parentheses(v_prime): # by Gemini # turns whole exp into 1 if OR 1 and into 0 if AND 0
    """
    Collapses parenthetical groups in a boolean vector if they match
    short-circuit conditions (AND 0, OR 1) at the top level of the group.
    
    Args:
        v_prime (list): A list of strings and integers (e.g., ['(', 'A', 'AND', 0, ')'])
    
    Returns:
        list: The modified list with short-circuits collapsed.
    """
    v = v_prime.copy()
    
    # We use a while loop because we modify the list length in place
    i = 0
    while i < len(v):
        if v[i] == "(":
            start = i
            
            # 1. Find the MATCHING closing parenthesis
            # We cannot just use .index(")") because that might catch a nested one.
            balance = 1
            end = -1
            for k in range(i + 1, len(v)):
                if v[k] == "(":
                    balance += 1
                elif v[k] == ")":
                    balance -= 1
                
                if balance == 0:
                    end = k
                    break
            
            # If no matching paren found (malformed), just move on
            if end == -1:
                i += 1
                continue

            # 2. Scan INSIDE the identified range [start+1 : end]
            # We look for short circuits, but we must SKIP nested parentheses
            j = start + 1
            found_short_circuit = None # Can be 0 or 1
            
            while j < end:
                token = v[j]
                
                # If we hit a nested open paren, skip this entire nested block
                if token == "(":
                    nested_balance = 1
                    while j < end and nested_balance > 0:
                        j += 1
                        if v[j] == "(": nested_balance += 1
                        elif v[j] == ")": nested_balance -= 1
                    # j is now at the closing paren of the nested group
                    j += 1
                    continue

                # CHECK FOR: AND 0
                # We check if token is 0, and if it has an AND neighbor
                if token == '0':
                    prev_is_and = (j > start + 1 and v[j-1] == "AND")
                    next_is_and = (j + 1 < end and v[j+1] == "AND")
                    
                    if prev_is_and or next_is_and:
                        found_short_circuit = '0'
                        break

                # CHECK FOR: OR 1
                # We check if token is 1, and if it has an OR neighbor
                if token == '1':
                    prev_is_or = (j > start + 1 and v[j-1] == "OR")
                    next_is_or = (j + 1 < end and v[j+1] == "OR")
                    
                    if prev_is_or or next_is_or:
                        found_short_circuit = '1'
                        break
                
                j += 1

            # 3. Apply Collapse if found
            if found_short_circuit is not None:
                # Replace everything from start to end (inclusive) with the result
                v[start:end+1] = [found_short_circuit]
                # Reset cursor to start to re-evaluate surrounding context if needed
                # (e.g., ( (A) AND 0 ) AND 0 might need two passes, checking from start is safest)
                i = 0 
            else:
                i += 1
        else:
            i += 1
            
    return v

def remove_redundant_nested_parentheses(v_prime): # by ChatGPT # turns ( ( A ) ) into ( A )
    v = v_prime.copy()
    """
    In-place: collapse only exact redundant nested parentheses of the form
      ( ( ... ) )
    into
      ( ... )

    Safety: only removes when the inner '(''s matching ')' is immediately
    followed by the outer ')'. This avoids removing '(' in cases like
      ( ( A OR B ) AND C )
    which must be preserved.
    """
    changed = True
    while changed:
        changed = False
        i = 0
        while i < len(v) - 1:
            # look for "( ("
            if v[i] == "(" and v[i+1] == "(":
                # find matching close for the inner '(' at position i+1
                depth = 0
                j = i + 1
                matched = -1
                while j < len(v):
                    if v[j] == "(":
                        depth += 1
                    elif v[j] == ")":
                        depth -= 1
                        if depth == 0:
                            matched = j
                            break
                    j += 1

                # matched is index of inner ')' if found
                # check that the token right after inner ')' is an outer ')'
                if matched != -1 and matched + 1 < len(v) and v[matched + 1] == ")":
                    # remove the outermost redundant pair:
                    # delete v[i] (the first '(') and v[matched + 1] (the corresponding outer ')')
                    # delete the later index first to avoid shifting problem
                    del v[matched + 1]
                    del v[i]
                    changed = True
                    # after deletion, stay at same i to re-check nested chains e.g. (((X))) 
                    continue
                else:
                    # not a simple redundant nested pair; advance
                    i += 1
            else:
                i += 1
    return v

def remove_redundant_nested_parentheses2(v_prime): # function that makes ( 0 ) into 0 and ( 1 ) into 1
    v = v_prime.copy()
    i = 0
    while i < len(v) - 2:
        if v[i] == "(" and v[i+2] == ")":
            if v[i+1] == '0' or v[i+1] == '1':
                # remove the parentheses
                v[i:i+3] = [v[i+1]]
                continue
        i += 1
    return v

def clean_andor_parentheses(v): # function that cleans things like AND ) and makes it just be )
    v_prime = v.copy()
    i = 0
    
    # Loop until the second to last element (since we check i+1)
    while i < len(v_prime) - 1:
        
        current_token = v_prime[i]
        next_token = v_prime[i+1]
        
        if current_token in ("AND", "OR") and next_token == ")":
            del v_prime[i]
            # Do not advance i; check the new token at this index

        elif current_token == "(" and next_token in ("AND", "OR"):
            del v_prime[i+1]
            # Do not advance i; check if another operator slid into place
            
        else:
            i += 1
            
    return v_prime
    

def clean_vector(v):
    v_prime = v.copy()
    
    while True: # repeat cleaning until no changes
        comparison_vector = v_prime.copy()
        
        v_prime = clean_andor_parentheses(v_prime)
        v_prime = remove_redundant_nested_parentheses2(v_prime)
        v_prime = clean_constants(v_prime)
        v_prime = collapse_parentheses(v_prime)
        v_prime = remove_redundant_nested_parentheses(v_prime)

        if comparison_vector == v_prime:
            break
    return v_prime


In [1058]:
def K(b, w, t, bv): # Can I learn word w from book b at time t, having read all books in bv?
    # bv for book vector: bv[0] was read at t=0, bv[1] at t=1...

    if b in bv: return ['0']
    # --- Change Reread ---

    # This stops Rereads from happening, 
    # if we have read b before (if its in bv), than we cant read it now

    # return [f"K({b},{w},{t},{bv})"]

    # checking for already known values of K with book b
    base_K = f"K({b},{w},0,[])"
    if base_K in set_of_ks:
        return set_of_ks[base_K]
    
    base_K = f"K({b},{w},{t},{bv})"
    if base_K in set_of_ks:
        return set_of_ks[base_K]
    
    # checking for already known values of K with books in bv
    for book in bv:
        base_K = f"K({book},{w},0,[])"
        if base_K in set_of_ks:
            return set_of_ks[base_K]
    
    # check if book b has w
    if w not in v_of_vocab_sets[b]:
        # w is not to be found in book b

        if bv == []:
            # we havent read any books yet (empty bv) and 
            # book b doesnt have w in it

            # So, we cant learn it through book b, and return 0
            return ['0']
        
        # maybe we could have learned it in a past book

        # So, we can go back in the history of books (bv)
        return K(bv[-1],w,t-1,bv[:-1])


    v_prime = []

    v_of_ors = []

    for s_d in range(len(B[b])):
        
        if w in B[b][s_d]:
            if len(B[b][s_d]) == 1: 
                # w is alone in s_d, so we learned w in this book
                # So, we can return a 1, cause we will learn w in this book b
                return ['1']

            # if w is not alone, we need to know
            # all other words omega
            v_of_ands = []
            for omega in B[b][s_d]:

                if omega == w:
                    continue
                
                v_of_ands.append(k(b, s_d, omega, t))
            v_of_ands = andify(v_of_ands) 
            # Make the AND of all ks of the words in s_d

            v_of_ors.append(v_of_ands)
    v_prime = orify(v_of_ors)
    # Make the OR of knowing w in all sentences in book b
    
    # add result to set_of_ks if not there yet
    base_K = f"K({b},{w},{t},{bv})"
    if base_K not in set_of_ks:
        set_of_ks[base_K] = v_prime

    return v_prime

    return [f"K({b},{w},{t},{bv})"]



In [1059]:
def K_reread(b, w, t): # Can I learn word w from book b at time t?
    # This K is the older version, which allows rereads, 
    # but hopefully can provide more aggregation, leaving the reread to be dealt by another constraint

    # checking for already known values of K with book b
    # base_K = f"K_reread({b},{w},0)"
    # if base_K in set_of_ks:
    #     if (w == 0):
    #         print( f"1st set check in K_reread({b},{w},{t}) = {set_of_ks[base_K]}")
    #     return set_of_ks[base_K]
    
    base_K = f"K_reread({b},{w},{t})"
    if base_K in set_of_ks:
        if (w == 0):
            print( f"2nd set check in K_reread({b},{w},{t}) = {set_of_ks[base_K]}")
        return set_of_ks[base_K]
    
    # check if book b has w
    if w not in v_of_vocab_sets[b]:
        # w is not to be found in book b

        if t == 0:
            # we read no books yet (t = 0) and 
            # book b doesnt have w in it

            # So, we cant learn it through book b, and return 0
            return ['0']
        
        # maybe we could have learned it in a past book

        # So, we check if we knew it at t-1

        # -------- 2 --------

        v_prime = [f"l[{w}][{t-1}]"]
        # v_prime = l[w][t-1]
        # ------ Change l ------

        # Add this value to the set_of_ks
        base_K = f"K_reread({b},{w},{t})"
        if base_K not in set_of_ks:
            if (w == 0):
                print( f"1st set add in K_ reread({b},{w},{t}) = {v_prime}")
            set_of_ks[base_K] = v_prime

        return v_prime
    
        
    
    # At this point, we know:
    #   1. Word w is in b
    #   2. Word w is not alone in a sentence in b (else it would be in vocab)
    
    # 99% of whats here on is the same as the K() which has bv. Only difference is when adding the value of it to the set_of_ks

    v_prime = []

    v_of_ors = []

    for s_d in range(len(B[b])):
        
        if w in B[b][s_d]:

            # This 'if' will never be entered because of "At this point, we know:" point 2
            if len(B[b][s_d]) == 1: 
                # w is alone in s_d, so we learned w in this book
                # So, we can return a 1, cause we will learn w in this book b
                return ['1']

            # if w is not alone, we need to know
            # all other words omega
            v_of_ands = []
            for omega in B[b][s_d]:

                if omega == w:
                    continue
                
                v_of_ands.append(k(b, s_d, omega, t))
            v_of_ands = andify(v_of_ands) 
            # Make the AND of all ks of the words in s_d

            v_of_ors.append(v_of_ands)
    v_prime = orify(v_of_ors)
    # Make the OR of knowing w in all sentences in book b
    
    # add result to set_of_ks if not there yet
    base_K = f"K_reread({b},{w},{t})"
    if base_K not in set_of_ks:
        if (w == 0):
            print( f"2nd set add in K_reread({b},{w},{t}) = {v_prime}")
        set_of_ks[base_K] = v_prime

    return v_prime


In [1060]:
def K(b, w, t, bv): # Can I learn word w from book b at time t, having read all books in bv?
    # bv for book vector: bv[0] was read at t=0, bv[1] at t=1...

    if b in bv: return ['0']
    # --- Change Reread ---

    # This stops Rereads from happening, 
    # if we have read b before (if its in bv), than we cant read it now

    # return [f"K({b},{w},{t},{bv})"]

    # checking for already known values of K with book b
    base_K = f"K({b},{w},0,[])"
    if base_K in set_of_ks:
        return set_of_ks[base_K]
    
    base_K = f"K({b},{w},{t},{bv})"
    if base_K in set_of_ks:
        return set_of_ks[base_K]
    
    # checking for already known values of K with books in bv
    for book in bv:
        base_K = f"K({book},{w},0,[])"
        if base_K in set_of_ks:
            return set_of_ks[base_K]
    
    # check if book b has w
    if w not in v_of_vocab_sets[b]:
        # w is not to be found in book b

        if bv == []:
            # we read any books yet (empty bv) and 
            # book b doesnt have w in it

            # So, we cant learn it through book b, and return 0
            return ['0']
        
        # maybe we could have learned it in a past book

        # So, we can go back in the history of books (bv)
        return K(bv[-1],w,t-1,bv[:-1])


    v_prime = []

    v_of_ors = []

    for s_d in range(len(B[b])):
        
        if w in B[b][s_d]:
            if len(B[b][s_d]) == 1: 
                # w is alone in s_d, so we learned w in this book
                # So, we can return a 1, cause we will learn w in this book b
                return ['1']

            # if w is not alone, we need to know
            # all other words omega
            v_of_ands = []
            for omega in B[b][s_d]:

                if omega == w:
                    continue
                
                v_of_ands.append(k(b, s_d, omega, t))
            v_of_ands = andify(v_of_ands) 
            # Make the AND of all ks of the words in s_d

            v_of_ors.append(v_of_ands)
    v_prime = orify(v_of_ors)
    # Make the OR of knowing w in all sentences in book b
    
    # add result to set_of_ks if not there yet
    base_K = f"K({b},{w},{t},{bv})"
    if base_K not in set_of_ks:
        set_of_ks[base_K] = v_prime

    return v_prime

    return [f"K({b},{w},{t},{bv})"]



In [1061]:
# Past K()s

{
# def K(b, w, t): # Is 1 if you learn word w reading book b at time t
#     # return [f"K({b},{w},{t})"]
#     #delete later?# global v
#     v_prime = []
#     v_prime.append("(")

#     # Now s_d goes from 0 to len(B[b]) - 1
#     for s_d in range(len(B[b])):

#         if w in B[b][s_d]:
#             if len(B[b][s_d]) == 1: 
#                 # w is alone in s_d, so we learned w in this book
#                 # So, we can return a 1, cause we will learn w in this book b
#                 return [1]

#             v_prime.append("(")

#             for omega in B[b][s_d]:
#                 if omega == w:
#                     continue

#                 v_prime += k(b, s_d, omega, t)
#                 v_prime.append("AND")
#             if v_prime[-1] == "AND": del v_prime[-1]

#             v_prime.append(")")
#             v_prime.append("OR")
#     if v_prime[-1] == "OR": del v_prime[-1]

#     v_prime.append(")")

#     if v_prime == ["(",")"]:
#         # we never found w in book b
#         # So, we can return a 0, cause we won't learn w in this book b
#         return [0]   
   
#     return v_prime
}



(
# def K(b, w, t): # Is 1 if you learn word w reading book b at time t
#     # return [f"K({b},{w},{t})"]
#     v_prime = []

#     # Now s_d goes from 0 to len(B[b]) - 1
#     for s_d in range(len(B[b])):

#         if w in B[b][s_d]:
#             if len(B[b][s_d]) == 1: 
#                 # w is alone in s_d, so we learned w in this book
#                 # So, we can return a 1, cause we will learn w in this book b
#                 return 1

#             vector_of_ands = []
#             for omega in B[b][s_d]:
#                 if omega == w:
#                     continue

#                 vector_of_ands += [k(b, s_d, omega, t)]

#                 # v_prime += k(b, s_d, omega, t)
#                 # v_prime.append("AND")
#             vector_of_ands = andify(vector_of_ands)
#             v_prime += [vector_of_ands]

#     if v_prime == []:
#         # we never found w in book b
#         # So, we can return a 0, cause we won't learn w in this book b
#         return 0

#     v_prime = orify(v_prime)


#     return v_prime
)
None

In [1062]:
def k(b, s_d, omega, t):

    # return [f"k({b},{s_d},{omega},{t})"]

    
    if len(B[b][s_d]) == 1 and B[b][s_d][0] == omega:
        # omega is the only word in sentence s_d of book b
        
        # I THINK THIS WHOLE SECTION OF THE DICT IS BEING USELESS
        # AFTER ALL, WE'RE NEVER CHECKING IN THE SET
        #     maybe we can do this by adding only in iterate_B and
        #     here just checking instead of doing this initial condition. 
        #     But i think its just waste of memory if we do that. 
        #     Only add to the set if you dont want too many iterations of k inside of itself

        # Add its value to the dictionary,
        # if not there yet
        base_k = f"k({b},{s_d},{omega})" 
        # base_k does not depend on t, since represents if omega is alone in s_d of b
        
        if base_k not in set_of_ks:
            set_of_ks[base_k] = ['1']
        


        # So, we learn omega reading book b at s_d. 
        # Hence, returning 1 for value of k
        return ['1']
    

    
    if s_d == 0: 
        if t == 0: return ['0'] 
        # if we are at the starting s_d and starting book
        # and s_d has more than 1 word, we cant learn it
        # so we return [0]
        
        # if l[omega][t-1] == 0 or l[omega][t-1] == 1:
        if l[omega][t-1] == ['0'] or l[omega][t-1] == ['1']:
            # if l[omega][t-1] is a single value, 
            # we can return it as the value to simplify later
            return l[omega][t-1]

        # if first sentence, we can only look back at t-1
        # return l[omega][t-1]
        return [f"l[{omega}][{t-1}]"]
        # --- Change l ---
    
    v_prime = []

    v_of_ors = []

    # EITHER we knew omega in the last sentence (s_d-1)
    v_of_ors.append(k(b, s_d-1, omega, t))
    # v_prime += k(b, s_d-1, omega, t) + ["OR"] 

    # OR if (omega is in s_d) we know ALL the other words w 
    # in the same s_d of book b ( make an AND of k(b,s_d-1,w,t) )
    # If (omega not in s_d) we can just rely on knowing it in s_d-1
    if omega in B[b][s_d]:
        v_of_ands = []

        for w in B[b][s_d]:
            if w == omega:
                continue

            v_of_ands.append(k(b, s_d-1, w, t))

        v_of_ands = andify(v_of_ands)
        v_of_ors.append(v_of_ands)
    
    v_prime = orify(v_of_ors)


    return v_prime

In [1063]:
# Books
#  Creating the Modelling


# B = [ 
#       [ [1] ] ,                          # b0
#       [ [0,1] ] ,                        # b1
#       [ [0,3], [2], [4,3,5], [4,0] ] ,   # b2
#       [ [0,3], [2], [4,3,5], [4,0] ] ,   # b3
#     ]

B = [ # good example for testing rereads
      [ [1,2,3,4],[1,2,3],[1,2],[1] ] ,                          # b0
      [ [5] ] ,                        # b1
      [ [0] ] ,   # b2
      [ [5], [1,2,0] ] ,   # b3
    ]

iterate_B(B) # puts vaues in set_of_ks

n_of_unknown_words = 6 #max number in B - 1

n_of_books_to_read = 4 # At most the amount of books; max value t+1 can have.

n_of_books = len(B)
# L = 

In [1064]:
# See later (full_build())
    # def full_build():
    #     for b in range(len(B)):
    #         for s_d in range(len(B[b])):
    #             for w in B[b][s_d]:
    #                 for t in range(n_of_books_to_read):

    #                     L[][][]    
                        
                    
                    


    # L = [w][t][b] of 
    #   n° of words BY
    #   n° of books to read (can be less than amount of books)) BY
    #   n° of books

    # L = [[[[] for b in range(len(B))] for t in range(n_of_books_to_read)] for w in range(n_of_unknown_words)]

    # inside L[w][t][b] there will be K(b,w,t)
    # in each l[w][t] we will do OR between all b

In [1065]:
def K_creator(b_first,w,bv):
     # rearanges elements to have whats necessary for
     # calculating K with the arguments K supports
     # This is because t for instance is useless now that we have bv as a history of books.
     # Even b is useless, it could just be part of bv. 
     # So here we adapt to the past version
     t = len(bv)

     if t == 0: return K(b_first,w,t,bv) 
     # bv is empty, so we dont need to change anything in its elements' order

     b_last = bv[-1]
     new_bv = bv.copy()
     del new_bv[-1]
     new_bv = [b_first] + new_bv

     return K(b_last,w,t,new_bv)

In [1066]:
def xK(b,w,t,bv):
    # Creates the structure of "(x[b] == t) AND" together with 
    # all other possibilities after until getting to K

    # if t != len(bv): raise ValueError("xK's t and bv not compatible")
    if b in bv: return ['0']
    # --- Change Reread ---
    
    if t == 0: return K_creator(b,w,bv) 
    # remember K_creator returns K, which is a vector. So no [] needed here
    
    v_prime = []
    v_of_ors = []

    for book in range(n_of_books):
        
        v_of_ands =[]

        v_of_ands.append(vector_x_eq_int(book, t-1))
        v_of_ands.append(xK(book,w,t-1,[b] + bv))
        v_of_ands = andify(v_of_ands)
        # andify([[(x[book] == t-1)],xK]) -> 
        # [(,(,(x[book] == t-1),),"AND",(,xK_contents,),)]

        v_of_ors.append(v_of_ands)
        
    v_prime = orify(v_of_ors)

    return v_prime

In [1067]:
l = [[[] for t in range(n_of_books_to_read)] for w in range(n_of_unknown_words)]

for t in range(n_of_books_to_read):
    for w in range(n_of_unknown_words):
        
        
        
        v_of_ors1 = []

        # Create "l[w][t-1] OR"
        if t != 0:
            if l[w][t-1] != [0]:

                v_of_ors1.append([f"l[{w}][{t-1}]"])
                # v_of_ors1.append(l[w][t-1])
                # --- Change l ---

        
        v_prime = []
        v_of_ors2 = []

        for b in range(n_of_books):

            v_of_ands = []

            v_of_ands.append(vector_x_eq_int(b,t))
            
            # v_of_ands.append(xK(b,w,t,[]))
            v_of_ands.append(K_reread(b,w,t))
            # --- Change K ---


            v_of_ands = andify(v_of_ands)

            v_of_ors2.append(v_of_ands)
        
        v_of_ors2 = orify(v_of_ors2)

        v_of_ors1.append(v_of_ors2)

        v_prime = orify(v_of_ors1)

        l[w][t] = v_prime
        # l[w][t] = clean_vector(l[w][t])
        # l[w][t] = clean_vector_sympy(l[w][t],n_of_books_to_read,n_of_unknown_words,n_of_books)



2nd set check in K_reread(2,0,0) = ['1']
2nd set add in K_reread(3,0,0) = ['(', '(', '(', '0', ')', '|', '(', '(', '0', ')', '&', '(', '0', ')', ')', ')', '&', '(', '(', '0', ')', '|', '(', '(', '0', ')', '&', '(', '0', ')', ')', ')', ')']
1st set add in K_ reread(0,0,1) = ['l[0][0]']
1st set add in K_ reread(1,0,1) = ['l[0][0]']
2nd set add in K_reread(3,0,1) = ['(', '(', '(', 'l[1][0]', ')', '|', '(', '(', 'l[2][0]', ')', '&', '(', 'l[0][0]', ')', ')', ')', '&', '(', '(', 'l[2][0]', ')', '|', '(', '(', 'l[1][0]', ')', '&', '(', 'l[0][0]', ')', ')', ')', ')']
1st set add in K_ reread(0,0,2) = ['l[0][1]']
1st set add in K_ reread(1,0,2) = ['l[0][1]']
2nd set add in K_reread(3,0,2) = ['(', '(', '(', 'l[1][1]', ')', '|', '(', '(', 'l[2][1]', ')', '&', '(', 'l[0][1]', ')', ')', ')', '&', '(', '(', 'l[2][1]', ')', '|', '(', '(', 'l[1][1]', ')', '&', '(', 'l[0][1]', ')', ')', ')', ')']
1st set add in K_ reread(0,0,3) = ['l[0][2]']
1st set add in K_ reread(1,0,3) = ['l[0][2]']
2nd set add in

In [1078]:
# Results Display
for t in range(n_of_books_to_read): # n_of_books_to_read
    for w in range(n_of_unknown_words): # n_of_unknown_words
        print(f"l[{w}][{t}]:")
        print("     ",end="")
        # print_vector(l[w][t])
        # print(l[w][t])

        # print()
        print(clean_vector_sympy(l[w][t],n_of_books_to_read,n_of_unknown_words,n_of_books))
        
        print()

# print_vector(l[3][1])

l[0][0]:
     Eq(X[2], 0)

l[1][0]:
     Eq(X[0], 0)

l[2][0]:
     False

l[3][0]:
     False

l[4][0]:
     False

l[5][0]:
     Eq(X[1], 0) | Eq(X[3], 0)

l[0][1]:
     L[0][0] | Eq(X[2], 1) | (L[1][0] & L[2][0] & Eq(X[3], 1))

l[1][1]:
     L[1][0] | Eq(X[0], 1) | (L[0][0] & L[2][0] & Eq(X[3], 1))

l[2][1]:
     L[2][0] | (L[1][0] & Eq(X[0], 1)) | (L[0][0] & L[1][0] & Eq(X[3], 1))

l[3][1]:
     L[3][0] | (L[1][0] & L[2][0] & Eq(X[0], 1))

l[4][1]:
     L[4][0] | (L[1][0] & L[2][0] & L[3][0] & Eq(X[0], 1))

l[5][1]:
     L[5][0] | Eq(X[1], 1) | Eq(X[3], 1)

l[0][2]:
     L[0][1] | Eq(X[2], 2) | (L[1][1] & L[2][1] & Eq(X[3], 2))

l[1][2]:
     L[1][1] | Eq(X[0], 2) | (L[0][1] & L[2][1] & Eq(X[3], 2))

l[2][2]:
     L[2][1] | (L[1][1] & Eq(X[0], 2)) | (L[0][1] & L[1][1] & Eq(X[3], 2))

l[3][2]:
     L[3][1] | (L[1][1] & L[2][1] & Eq(X[0], 2))

l[4][2]:
     L[4][1] | (L[1][1] & L[2][1] & L[3][1] & Eq(X[0], 2))

l[5][2]:
     L[5][1] | Eq(X[1], 2) | Eq(X[3], 2)

l[0][3]:
     L[0][2] 

In [ ]:
# Rascunho em Python 4

l[0][0]:
     Eq(X[2], 0)

l[1][0]:
     Eq(X[0], 0)

l[2][0]:
     False

l[3][0]:
     False

l[4][0]:
     False

l[5][0]:
     Eq(X[1], 0) | Eq(X[3], 0)

l[0][1]:
     L[0][0] | Eq(X[2], 1) | (L[1][0] & L[2][0] & Eq(X[3], 1))

l[1][1]:
     L[1][0] | Eq(X[0], 1) | (L[0][0] & L[2][0] & Eq(X[3], 1))

l[2][1]:
     L[2][0] | (L[1][0] & Eq(X[0], 1)) | (L[0][0] & L[1][0] & Eq(X[3], 1))

l[3][1]:
     L[3][0] | (L[1][0] & L[2][0] & Eq(X[0], 1))

l[4][1]:
     L[4][0] | (L[1][0] & L[2][0] & L[3][0] & Eq(X[0], 1))

l[5][1]:
     L[5][0] | Eq(X[1], 1) | Eq(X[3], 1)

l[0][2]:
     L[0][1] | Eq(X[2], 2) | (L[1][1] & L[2][1] & Eq(X[3], 2))

l[1][2]:
     L[1][1] | Eq(X[0], 2) | (L[0][1] & L[2][1] & Eq(X[3], 2))

l[2][2]:
     L[2][1] | (L[1][1] & Eq(X[0], 2)) | (L[0][1] & L[1][1] & Eq(X[3], 2))

l[3][2]:
     L[3][1] | (L[1][1] & L[2][1] & Eq(X[0], 2))

l[4][2]:
     L[4][1] | (L[1][1] & L[2][1] & L[3][1] & Eq(X[0], 2))

l[5][2]:
     L[5][1] | Eq(X[1], 2) | Eq(X[3], 2)

l[0][3]:
     L[0][2] | Eq(X[2], 3) | (L[1][2] & L[2][2] & Eq(X[3], 3))

l[1][3]:
     L[1][2] | Eq(X[0], 3) | (L[0][2] & L[2][2] & Eq(X[3], 3))

l[2][3]:
     L[2][2] | (L[1][2] & Eq(X[0], 3)) | (L[0][2] & L[1][2] & Eq(X[3], 3))

l[3][3]:
     L[3][2] | (L[1][2] & L[2][2] & Eq(X[0], 3))

l[4][3]:
     L[4][2] | (L[1][2] & L[2][2] & L[3][2] & Eq(X[0], 3))

l[5][3]:
     L[5][2] | Eq(X[1], 3) | Eq(X[3], 3)

SyntaxError: invalid syntax (874085803.py, line 3)

In [ ]:
# Rascunho em Python 3

l[2][1]: # K not expanded
     ( 
          ( l[2][0] ) OR 
          
          ( 
               ( x[0] == 1 ) AND 
               ( ( ( x[1] == 0 ) AND ( K(0,2,1,[1]) ) ) OR ( ( x[2] == 0 ) AND ( K(0,2,1,[2]) ) ) OR ( ( x[3] == 0 ) AND ( K(0,2,1,[3]) ) ) ) 
          ) OR 
          
          ( 
               ( x[1] == 1 ) AND 
               ( ( ( x[0] == 0 ) AND ( K(1,2,1,[0]) ) ) OR ( ( x[2] == 0 ) AND ( K(1,2,1,[2]) ) ) OR ( ( x[3] == 0 ) AND ( K(1,2,1,[3]) ) ) ) 
          ) OR 
          
          ( 
               ( x[2] == 1 ) AND 
               ( ( ( x[0] == 0 ) AND ( K(2,2,1,[0]) ) ) OR ( ( x[1] == 0 ) AND ( K(2,2,1,[1]) ) ) OR ( ( x[3] == 0 ) AND ( K(2,2,1,[3]) ) ) ) 
          ) OR 
          
          ( 
               ( x[3] == 1 ) AND 
               ( ( ( x[0] == 0 ) AND ( K(3,2,1,[0]) ) ) OR ( ( x[1] == 0 ) AND ( K(3,2,1,[1]) ) ) OR ( ( x[2] == 0 ) AND ( K(3,2,1,[2]) ) ) ) 
          ) 
     )



l[2][1]: # k not expanded
     ( 
          ( l[2][0] ) OR 
          
          ( 
               ( x[0] == 1 ) AND 
               
               ( 
                    ( ( x[1] == 0 ) AND ( ( k(0,0,1,1) AND k(0,0,3,1) AND k(0,0,4,1) ) OR ( k(0,1,1,1) AND k(0,1,3,1) ) OR ( k(0,2,1,1) ) ) ) OR 
                    
                    ( ( x[2] == 0 ) AND ( ( k(0,0,1,1) AND k(0,0,3,1) AND k(0,0,4,1) ) OR ( k(0,1,1,1) AND k(0,1,3,1) ) OR ( k(0,2,1,1) ) ) ) OR 
                    
                    ( ( x[3] == 0 ) AND ( ( k(0,0,1,1) AND k(0,0,3,1) AND k(0,0,4,1) ) OR ( k(0,1,1,1) AND k(0,1,3,1) ) OR ( k(0,2,1,1) ) ) ) 
               ) 
          )
     )

l[2][1]:
     ( 
          ( x[0] == 1 ) AND 

          ( 
               ( ( x[1] == 0 ) AND ( l[1][0] ) ) OR 
               ( ( x[2] == 0 ) AND ( l[1][0] ) ) OR 
               ( ( x[3] == 0 ) AND ( l[1][0] ) ) 
          ) 
     )

l[2][1] ( x[0] == 1 ) AND ( x[1] == 0 ) AND ( l[1][0] )


l[1][0]:
     ( x[0] == 0 )

l[1][1]:
     ( 
          ( x[0] == 0 ) OR 
          ( ( x[0] == 1 ) AND ( ( x[1] == 0 ) OR ( x[2] == 0 ) OR ( x[3] == 0 ) ) ) OR 
          ( ( x[1] == 1 ) AND ( x[0] == 0 ) ) OR 
          ( ( x[2] == 1 ) AND ( x[0] == 0 ) ) OR 
          ( ( x[3] == 1 ) AND ( x[0] == 0 ) ) 
     )

l[1][1] 
     A or (B and (C or D or F)) or (G and A) or (H and A) or (I and A)
     A or (B and (C or D or F)) or ((G or H or I) and A)


l[2][2]:
     ( 
          ( l[2][1] ) OR 
          
          ( 
               ( x[0] == 2 ) AND 
               
               ( 
                    ( 
                         ( x[1] == 1 ) AND 
                         ( 
                              ( 
                                   ( x[2] == 0 ) AND 
                                   ( ( ( l[1][1] ) AND ( l[1][1] AND l[2][1] ) ) OR ( ( l[1][1] ) OR ( l[2][1] ) ) ) 
                              ) OR 

                              ( 
                                   ( x[3] == 0 ) AND 
                                   ( ( ( l[1][1] ) AND ( l[1][1] AND l[2][1] ) ) OR ( ( l[1][1] ) OR ( l[2][1] ) ) ) 
                              ) 
                         )
                    ) OR 
                    
                    ( 
                         ( x[2] == 1 ) AND 
                         ( 
                              ( ( x[1] == 0 ) AND ( ( ( l[1][1] ) AND ( l[1][1] AND l[2][1] ) ) OR ( ( l[1][1] ) OR ( l[2][1] ) ) ) ) OR 
                              
                              ( ( x[3] == 0 ) AND ( ( ( l[1][1] ) AND ( l[1][1] AND l[2][1] ) ) OR ( ( l[1][1] ) OR ( l[2][1] ) ) ) ) 
                         ) 
                    ) OR 
                    
                    ( 
                         ( x[3] == 1 ) AND 
                         ( 
                              ( ( x[1] == 0 ) AND ( ( ( l[1][1] ) AND ( l[1][1] AND l[2][1] ) ) OR ( ( l[1][1] ) OR ( l[2][1] ) ) ) ) OR 
                              
                              ( ( x[2] == 0 ) AND ( ( ( l[1][1] ) AND ( l[1][1] AND l[2][1] ) ) OR ( ( l[1][1] ) OR ( l[2][1] ) ) ) ) 
                         ) 
                    ) 
               ) 
          )
     )



SyntaxError: invalid syntax (3325965032.py, line 3)

In [ ]:
# Rascunho em Python 2

l[2][0]:
     ( ( ( x[0] == 0 ) AND ( K(0,2,0,[]) ) ) OR ( ( x[1] == 0 ) AND ( K(1,2,0,[]) ) ) OR ( ( x[2] == 0 ) AND ( K(2,2,0,[]) ) ) OR ( ( x[3] == 0 ) AND ( K(3,2,0,[]) ) ) ) 

l[2][1]:
     ( ( l[2][0] ) OR ( ( x[0] == 1 ) AND ( ( ( x[1] == 0 ) AND ( K(0,2,1,[1]) ) ) OR ( ( x[2] == 0 ) AND ( K(0,2,1,[2]) ) ) OR ( ( x[3] == 0 ) AND ( K(0,2,1,[3]) ) ) ) ) OR ( ( x[1] == 1 ) AND ( ( ( x[0] == 0 ) AND ( K(1,2,1,[0]) ) ) OR ( ( x[2] == 0 ) AND ( K(1,2,1,[2]) ) ) OR ( ( x[3] == 0 ) AND ( K(1,2,1,[3]) ) ) ) ) OR ( ( x[2] == 1 ) AND ( ( ( x[0] == 0 ) AND ( K(2,2,1,[0]) ) ) OR ( ( x[1] == 0 ) AND ( K(2,2,1,[1]) ) ) OR ( ( x[3] == 0 ) AND ( K(2,2,1,[3]) ) ) ) ) OR ( ( x[3] == 1 ) AND ( ( ( x[0] == 0 ) AND ( K(3,2,1,[0]) ) ) OR ( ( x[1] == 0 ) AND ( K(3,2,1,[1]) ) ) OR ( ( x[2] == 0 ) AND ( K(3,2,1,[2]) ) ) ) ) ) 

l[2][2]:
     ( 
          ( l[2][1] ) OR 
          
          ( ( x[0] == 2 ) AND 
               ( 
                    ( ( x[1] == 1 ) AND 
                         ( 
                              ( ( x[2] == 0 ) AND ( K(0,2,2,[2, 1]) ) ) OR 
                              ( ( x[3] == 0 ) AND ( K(0,2,2,[3, 1]) ) ) ) 
                    ) 
                         
                    OR 

                    ( ( x[2] == 1 ) AND ( ( ( x[1] == 0 ) AND ( K(0,2,2,[1, 2]) ) ) OR ( ( x[3] == 0 ) AND ( K(0,2,2,[3, 2]) ) ) ) ) OR 
                    
                    ( ( x[3] == 1 ) AND ( ( ( x[1] == 0 ) AND ( K(0,2,2,[1, 3]) ) ) OR ( ( x[2] == 0 ) AND ( K(0,2,2,[2, 3]) ) ) ) ) ) 
          ) 
                    
          OR 
          
          ( ( x[1] == 2 ) AND ( ( ( x[0] == 1 ) AND ( ( ( x[2] == 0 ) AND ( K(1,2,2,[2, 0]) ) ) OR ( ( x[3] == 0 ) AND ( K(1,2,2,[3, 0]) ) ) ) ) OR ( ( x[2] == 1 ) AND ( ( ( x[0] == 0 ) AND ( K(1,2,2,[0, 2]) ) ) OR ( ( x[3] == 0 ) AND ( K(1,2,2,[3, 2]) ) ) ) ) OR ( ( x[3] == 1 ) AND ( ( ( x[0] == 0 ) AND ( K(1,2,2,[0, 3]) ) ) OR ( ( x[2] == 0 ) AND ( K(1,2,2,[2, 3]) ) ) ) ) ) ) OR 
          
          ( ( x[2] == 2 ) AND ( ( ( x[0] == 1 ) AND ( ( ( x[1] == 0 ) AND ( K(2,2,2,[1, 0]) ) ) OR ( ( x[3] == 0 ) AND ( K(2,2,2,[3, 0]) ) ) ) ) OR ( ( x[1] == 1 ) AND ( ( ( x[0] == 0 ) AND ( K(2,2,2,[0, 1]) ) ) OR ( ( x[3] == 0 ) AND ( K(2,2,2,[3, 1]) ) ) ) ) OR ( ( x[3] == 1 ) AND ( ( ( x[0] == 0 ) AND ( K(2,2,2,[0, 3]) ) ) OR ( ( x[1] == 0 ) AND ( K(2,2,2,[1, 3]) ) ) ) ) ) ) OR 
          
          ( ( x[3] == 2 ) AND ( ( ( x[0] == 1 ) AND ( ( ( x[1] == 0 ) AND ( K(3,2,2,[1, 0]) ) ) OR ( ( x[2] == 0 ) AND ( K(3,2,2,[2, 0]) ) ) ) ) OR ( ( x[1] == 1 ) AND ( ( ( x[0] == 0 ) AND ( K(3,2,2,[0, 1]) ) ) OR ( ( x[2] == 0 ) AND ( K(3,2,2,[2, 1]) ) ) ) ) OR ( ( x[2] == 1 ) AND ( ( ( x[0] == 0 ) AND ( K(3,2,2,[0, 2]) ) ) OR ( ( x[1] == 0 ) AND ( K(3,2,2,[1, 2]) ) ) ) ) ) ) 
     ) 

l[2][3]:
     ( ( l[2][2] ) OR ( ( x[0] == 3 ) AND ( ( ( x[1] == 2 ) AND ( ( ( x[2] == 1 ) AND ( ( x[3] == 0 ) AND ( K(0,2,3,[3, 2, 1]) ) ) ) OR ( ( x[3] == 1 ) AND ( ( x[2] == 0 ) AND ( K(0,2,3,[2, 3, 1]) ) ) ) ) ) OR ( ( x[2] == 2 ) AND ( ( ( x[1] == 1 ) AND ( ( x[3] == 0 ) AND ( K(0,2,3,[3, 1, 2]) ) ) ) OR ( ( x[3] == 1 ) AND ( ( x[1] == 0 ) AND ( K(0,2,3,[1, 3, 2]) ) ) ) ) ) OR ( ( x[3] == 2 ) AND ( ( ( x[1] == 1 ) AND ( ( x[2] == 0 ) AND ( K(0,2,3,[2, 1, 3]) ) ) ) OR ( ( x[2] == 1 ) AND ( ( x[1] == 0 ) AND ( K(0,2,3,[1, 2, 3]) ) ) ) ) ) ) ) OR ( ( x[1] == 3 ) AND ( ( ( x[0] == 2 ) AND ( ( ( x[2] == 1 ) AND ( ( x[3] == 0 ) AND ( K(1,2,3,[3, 2, 0]) ) ) ) OR ( ( x[3] == 1 ) AND ( ( x[2] == 0 ) AND ( K(1,2,3,[2, 3, 0]) ) ) ) ) ) OR ( ( x[2] == 2 ) AND ( ( ( x[0] == 1 ) AND ( ( x[3] == 0 ) AND ( K(1,2,3,[3, 0, 2]) ) ) ) OR ( ( x[3] == 1 ) AND ( ( x[0] == 0 ) AND ( K(1,2,3,[0, 3, 2]) ) ) ) ) ) OR ( ( x[3] == 2 ) AND ( ( ( x[0] == 1 ) AND ( ( x[2] == 0 ) AND ( K(1,2,3,[2, 0, 3]) ) ) ) OR ( ( x[2] == 1 ) AND ( ( x[0] == 0 ) AND ( K(1,2,3,[0, 2, 3]) ) ) ) ) ) ) ) OR ( ( x[2] == 3 ) AND ( ( ( x[0] == 2 ) AND ( ( ( x[1] == 1 ) AND ( ( x[3] == 0 ) AND ( K(2,2,3,[3, 1, 0]) ) ) ) OR ( ( x[3] == 1 ) AND ( ( x[1] == 0 ) AND ( K(2,2,3,[1, 3, 0]) ) ) ) ) ) OR ( ( x[1] == 2 ) AND ( ( ( x[0] == 1 ) AND ( ( x[3] == 0 ) AND ( K(2,2,3,[3, 0, 1]) ) ) ) OR ( ( x[3] == 1 ) AND ( ( x[0] == 0 ) AND ( K(2,2,3,[0, 3, 1]) ) ) ) ) ) OR ( ( x[3] == 2 ) AND ( ( ( x[0] == 1 ) AND ( ( x[1] == 0 ) AND ( K(2,2,3,[1, 0, 3]) ) ) ) OR ( ( x[1] == 1 ) AND ( ( x[0] == 0 ) AND ( K(2,2,3,[0, 1, 3]) ) ) ) ) ) ) ) OR ( ( x[3] == 3 ) AND ( ( ( x[0] == 2 ) AND ( ( ( x[1] == 1 ) AND ( ( x[2] == 0 ) AND ( K(3,2,3,[2, 1, 0]) ) ) ) OR ( ( x[2] == 1 ) AND ( ( x[1] == 0 ) AND ( K(3,2,3,[1, 2, 0]) ) ) ) ) ) OR ( ( x[1] == 2 ) AND ( ( ( x[0] == 1 ) AND ( ( x[2] == 0 ) AND ( K(3,2,3,[2, 0, 1]) ) ) ) OR ( ( x[2] == 1 ) AND ( ( x[0] == 0 ) AND ( K(3,2,3,[0, 2, 1]) ) ) ) ) ) OR ( ( x[2] == 2 ) AND ( ( ( x[0] == 1 ) AND ( ( x[1] == 0 ) AND ( K(3,2,3,[1, 0, 2]) ) ) ) OR ( ( x[1] == 1 ) AND ( ( x[0] == 0 ) AND ( K(3,2,3,[0, 1, 2]) ) ) ) ) ) ) ) )

SyntaxError: invalid syntax (187261274.py, line 3)

In [ ]:
# Rascunho em Python 1
( # Rewrite 3
     
     # l[0][1]: 
     #      ( 
     #           ( (x[0] == 1) AND 
     #                ( (x[0] == 0) AND (K(0,0,1,[0]) OR K(0,0,0,[])) ) OR
     #                ( (x[1] == 0) AND (K(0,0,1,[1]) OR K(1,0,0,[])) ) OR
     #                ( (x[2] == 0) AND (K(0,0,1,[2]) OR K(2,0,0,[])) ) OR
     #                ( (x[3] == 0) AND (K(0,0,1,[3]) OR K(3,0,0,[])) ) )
               
     #           # ( (x[0] == 1) AND 
     #           #      ( ((x[0] == 0) AND K(0,0,1,[0])) OR ((x[0] == 0) AND K(0,0,0,[])) ) OR
     #           #      ( ((x[1] == 0) AND K(0,0,1,[1])) OR ((x[1] == 0) AND K(1,0,0,[])) ) OR
     #           #      ( ((x[2] == 0) AND K(0,0,1,[2])) OR ((x[2] == 0) AND K(2,0,0,[])) ) OR
     #           #      ( ((x[3] == 0) AND K(0,0,1,[3])) OR ((x[3] == 0) AND K(3,0,0,[])) ) )

               
     #           OR
               
     #           ( (x[1] == 1) AND (x[0] == 0) AND (K(0,0,0,[]) OR K(1,0,1,[0])) ) OR
     #           ( (x[1] == 1) AND (x[1] == 0) AND (K(1,0,0,[]) OR K(1,0,1,[1])) ) OR
     #           ( (x[1] == 1) AND (x[2] == 0) AND (K(2,0,0,[]) OR K(1,0,1,[2])) ) OR
     #           ( (x[1] == 1) AND (x[3] == 0) AND (K(3,0,0,[]) OR K(1,0,1,[3])) )
               
     #           OR



     #           # ( x[1] == 1 ) AND ( x[1] == 0 ) ... OR # ---------- WRONG ----------
     #      ) 
)

( # Rewrite 4
# l[0][1]: 
#      ( 
#           ( (x[1] == 1) AND (x[0] == 0) AND (K(1,0,1,[0])) ) OR
#           ( (x[1] == 1) AND (x[1] == 0) AND (K(1,0,1,[1])) ) OR
#           ( (x[1] == 1) AND (x[2] == 0) AND (K(1,0,1,[2])) ) OR
#           ( (x[1] == 1) AND (x[3] == 0) AND (K(1,0,1,[3])) )

#           # ( x[1] == 1 ) AND ( x[1] == 0 ) ... OR # ---------- WRONG ----------
#      ) 
)

( # Rewrite 5
# l[0][1]: 
#      ( 

#           ( (x[0] == 1) AND 
#                ( (x[0] == 0) AND (K(0,0,1,[0])) ) OR
#                ( (x[1] == 0) AND (K(0,0,1,[1])) ) OR
#                ( (x[2] == 0) AND (K(0,0,1,[2])) ) OR
#                ( (x[3] == 0) AND (K(0,0,1,[3])) ) )
          
#           OR

#           ( (x[1] == 1) AND 
#                ( (x[0] == 0) AND (K(1,0,1,[0])) ) OR
#                ( (x[1] == 0) AND (K(1,0,1,[1])) ) OR
#                ( (x[2] == 0) AND (K(1,0,1,[2])) ) OR
#                ( (x[3] == 0) AND (K(1,0,1,[3])) ) )

#           OR

#           ( (x[1] == 3) AND 
#                ( (x[0] == 2) AND (x[3] == 1) AND (x[2] == 0) AND (K_creator(2,0,[3,0,1])) ) OR
#                K(1,0,3,[2,3,0])


#                ( (x[1] == 0) AND (K(1,0,1,[1])) ) OR
#                ( (x[2] == 0) AND (K(1,0,1,[2])) ) OR
#                ( (x[3] == 0) AND (K(1,0,1,[3])) ) )

#           # ( x[1] == 1 ) AND ( x[1] == 0 ) ... OR # ---------- WRONG ----------
#      ) 
)

l[0][2]: # actual output, not cleaned
     ( 
          ( ( x[0] == 2 ) AND 
               ( ( x[0] == 1 ) AND 0 ) OR 
               
               ( ( x[1] == 1 ) AND 
                    ( ( x[0] == 0 ) AND 0 ) OR 
                    ( ( x[1] == 0 ) AND 0 ) OR 
                    ( ( x[2] == 0 ) AND K(0,0,2,[2, 1]) ) OR 
                    ( ( x[3] == 0 ) AND K(0,0,2,[3, 1]) ) ) 
               
               OR 
               
               ( ( x[2] == 1 ) AND ( ( x[0] == 0 ) AND 0 ) OR ( ( x[1] == 0 ) AND K(0,0,2,[1, 2]) ) OR ( ( x[2] == 0 ) AND 0 ) OR ( ( x[3] == 0 ) AND K(0,0,2,[3, 2]) ) ) OR 
               
               ( ( x[3] == 1 ) AND ( ( x[0] == 0 ) AND 0 ) OR ( ( x[1] == 0 ) AND K(0,0,2,[1, 3]) ) OR ( ( x[2] == 0 ) AND K(0,0,2,[2, 3]) ) OR ( ( x[3] == 0 ) AND 0 ) ) ) 
               
          OR 
          
          ( ( x[1] == 2 ) AND ( ( x[0] == 1 ) AND ( ( x[0] == 0 ) AND 0 ) OR ( ( x[1] == 0 ) AND 0 ) OR ( ( x[2] == 0 ) AND K(1,0,2,[2, 0]) ) OR ( ( x[3] == 0 ) AND K(1,0,2,[3, 0]) ) ) OR ( ( x[1] == 1 ) AND 0 ) OR ( ( x[2] == 1 ) AND ( ( x[0] == 0 ) AND K(1,0,2,[0, 2]) ) OR ( ( x[1] == 0 ) AND 0 ) OR ( ( x[2] == 0 ) AND 0 ) OR ( ( x[3] == 0 ) AND K(1,0,2,[3, 2]) ) ) OR ( ( x[3] == 1 ) AND ( ( x[0] == 0 ) AND K(1,0,2,[0, 3]) ) OR ( ( x[1] == 0 ) AND 0 ) OR ( ( x[2] == 0 ) AND K(1,0,2,[2, 3]) ) OR ( ( x[3] == 0 ) AND 0 ) ) ) OR ( ( x[2] == 2 ) AND ( ( x[0] == 1 ) AND ( ( x[0] == 0 ) AND 0 ) OR ( ( x[1] == 0 ) AND K(2,0,2,[1, 0]) ) OR ( ( x[2] == 0 ) AND 0 ) OR ( ( x[3] == 0 ) AND K(2,0,2,[3, 0]) ) ) OR ( ( x[1] == 1 ) AND ( ( x[0] == 0 ) AND K(2,0,2,[0, 1]) ) OR ( ( x[1] == 0 ) AND 0 ) OR ( ( x[2] == 0 ) AND 0 ) OR ( ( x[3] == 0 ) AND K(2,0,2,[3, 1]) ) ) OR ( ( x[2] == 1 ) AND 0 ) OR ( ( x[3] == 1 ) AND ( ( x[0] == 0 ) AND K(2,0,2,[0, 3]) ) OR ( ( x[1] == 0 ) AND K(2,0,2,[1, 3]) ) OR ( ( x[2] == 0 ) AND 0 ) OR ( ( x[3] == 0 ) AND 0 ) ) ) OR ( ( x[3] == 2 ) AND ( ( x[0] == 1 ) AND ( ( x[0] == 0 ) AND 0 ) OR ( ( x[1] == 0 ) AND K(3,0,2,[1, 0]) ) OR ( ( x[2] == 0 ) AND K(3,0,2,[2, 0]) ) OR ( ( x[3] == 0 ) AND 0 ) ) OR ( ( x[1] == 1 ) AND ( ( x[0] == 0 ) AND K(3,0,2,[0, 1]) ) OR ( ( x[1] == 0 ) AND 0 ) OR ( ( x[2] == 0 ) AND K(3,0,2,[2, 1]) ) OR ( ( x[3] == 0 ) AND 0 ) ) OR ( ( x[2] == 1 ) AND ( ( x[0] == 0 ) AND K(3,0,2,[0, 2]) ) OR ( ( x[1] == 0 ) AND K(3,0,2,[1, 2]) ) OR ( ( x[2] == 0 ) AND 0 ) OR ( ( x[3] == 0 ) AND 0 ) ) OR ( ( x[3] == 1 ) AND 0 ) ) 
     )

l[0][2]:
     ( 
          ( ( x[0] == 2 ) AND 
               ( ( ( x[1] == 1 ) AND ( ( ( x[2] == 0 ) AND ( K(0,0,2,[2, 1]) ) ) OR ( ( x[3] == 0 ) AND ( K(0,0,2,[3, 1]) ) ) ) ) OR 
                
                ( ( x[2] == 1 ) AND ( ( ( x[1] == 0 ) AND ( K(0,0,2,[1, 2]) ) ) OR ( ( x[3] == 0 ) AND ( K(0,0,2,[3, 2]) ) ) ) ) OR 

                ( ( x[3] == 1 ) AND ( ( ( x[1] == 0 ) AND ( K(0,0,2,[1, 3]) ) ) OR ( ( x[2] == 0 ) AND ( K(0,0,2,[2, 3]) ) ) ) ) ) 
          ) 
               
          OR 

          ( ( x[1] == 2 ) AND ( ( ( x[0] == 1 ) AND ( ( ( x[2] == 0 ) AND ( K(1,0,2,[2, 0]) ) ) OR ( ( x[3] == 0 ) AND ( K(1,0,2,[3, 0]) ) ) ) ) OR ( ( x[2] == 1 ) AND ( ( ( x[0] == 0 ) AND ( K(1,0,2,[0, 2]) ) ) OR ( ( x[3] == 0 ) AND ( K(1,0,2,[3, 2]) ) ) ) ) OR ( ( x[3] == 1 ) AND ( ( ( x[0] == 0 ) AND ( K(1,0,2,[0, 3]) ) ) OR ( ( x[2] == 0 ) AND ( K(1,0,2,[2, 3]) ) ) ) ) ) ) OR 

          ( ( x[2] == 2 ) AND ( ( ( x[0] == 1 ) AND ( ( ( x[1] == 0 ) AND ( K(2,0,2,[1, 0]) ) ) OR ( ( x[3] == 0 ) AND ( K(2,0,2,[3, 0]) ) ) ) ) OR ( ( x[1] == 1 ) AND ( ( ( x[0] == 0 ) AND ( K(2,0,2,[0, 1]) ) ) OR ( ( x[3] == 0 ) AND ( K(2,0,2,[3, 1]) ) ) ) ) OR ( ( x[3] == 1 ) AND ( ( ( x[0] == 0 ) AND ( K(2,0,2,[0, 3]) ) ) OR ( ( x[1] == 0 ) AND ( K(2,0,2,[1, 3]) ) ) ) ) ) ) OR 

          ( ( x[3] == 2 ) AND ( ( ( x[0] == 1 ) AND ( ( ( x[1] == 0 ) AND ( K(3,0,2,[1, 0]) ) ) OR ( ( x[2] == 0 ) AND ( K(3,0,2,[2, 0]) ) ) ) ) OR ( ( x[1] == 1 ) AND ( ( ( x[0] == 0 ) AND ( K(3,0,2,[0, 1]) ) ) OR ( ( x[2] == 0 ) AND ( K(3,0,2,[2, 1]) ) ) ) ) OR ( ( x[2] == 1 ) AND ( ( ( x[0] == 0 ) AND ( K(3,0,2,[0, 2]) ) ) OR ( ( x[1] == 0 ) AND ( K(3,0,2,[1, 2]) ) ) ) ) ) ) 
     )

l[0][1]: # actual output, not cleaned 2
     ( 
          ( ( x[0] == 1 ) AND 
               ( ( x[0] == 0 ) AND 0 ) OR 
               ( ( x[1] == 0 ) AND K(0,0,1,[1]) ) OR 
               ( ( x[2] == 0 ) AND K(0,0,1,[2]) ) OR 
               ( ( x[3] == 0 ) AND K(0,0,1,[3]) ) ) 
          
          OR 
               
          ( ( x[1] == 1 ) AND ( ( x[0] == 0 ) AND K(1,0,1,[0]) ) OR ( ( x[1] == 0 ) AND 0 ) OR ( ( x[2] == 0 ) AND K(1,0,1,[2]) ) OR ( ( x[3] == 0 ) AND K(1,0,1,[3]) ) ) OR ( ( x[2] == 1 ) AND ( ( x[0] == 0 ) AND K(2,0,1,[0]) ) OR ( ( x[1] == 0 ) AND K(2,0,1,[1]) ) OR ( ( x[2] == 0 ) AND 0 ) OR ( ( x[3] == 0 ) AND K(2,0,1,[3]) ) ) OR ( ( x[3] == 1 ) AND ( ( x[0] == 0 ) AND K(3,0,1,[0]) ) OR ( ( x[1] == 0 ) AND K(3,0,1,[1]) ) OR ( ( x[2] == 0 ) AND K(3,0,1,[2]) ) OR ( ( x[3] == 0 ) AND 0 ) ) )


l[0][1]: # actual output, not cleaned 1
     (
          ( ( x[0] == 1 ) AND 
               ( ( x[0] == 1 ) AND 0 ) OR 
               ( ( x[1] == 1 ) AND K(0,0,1,[1]) ) OR 
               ( ( x[2] == 1 ) AND K(0,0,1,[2]) ) OR 
               ( ( x[3] == 1 ) AND K(0,0,1,[3]) ) ) 


          OR 

          
          ( ( x[1] == 1 ) AND 
               ( ( x[0] == 1 ) AND K(1,0,1,[0]) ) OR 
               ( ( x[1] == 1 ) AND 0 ) OR 
               ( ( x[2] == 1 ) AND K(1,0,1,[2]) ) OR 
               ( ( x[3] == 1 ) AND K(1,0,1,[3]) ) ) 
               
          
          OR 

          
          ( ( x[2] == 1 ) AND 
               ( ( x[0] == 1 ) AND K(2,0,1,[0]) ) OR 
               ( ( x[1] == 1 ) AND K(2,0,1,[1]) ) OR 
               ( ( x[2] == 1 ) AND 0 ) OR 
               ( ( x[3] == 1 ) AND K(2,0,1,[3]) ) ) 


          OR 
          

          ( ( x[3] == 1 ) AND 
               ( ( x[0] == 1 ) AND K(3,0,1,[0]) ) OR 
               ( ( x[1] == 1 ) AND K(3,0,1,[1]) ) OR 
               ( ( x[2] == 1 ) AND K(3,0,1,[2]) ) OR 
               ( ( x[3] == 1 ) AND 0 ) ) 
     )









def K_creator(b_first,w,bv):
     t = len(bv)

     if t == 0: return K(b_first,w,t,bv)

     b_last = bv[-1]
     new_bv = bv.copy()
     del new_bv[-1]
     new_bv = [b_first] + new_bv

     return K(b_last,w,t,new_bv)







l[0][1]:
     ( 
          ( ( x[0] == 0 ) AND 0 ) OR 
          ( ( x[1] == 0 ) AND k(1,0,1,0) ) OR 
          ( ( x[2] == 0 ) AND ( k(2,0,3,0) OR k(2,3,4,0) ) ) OR 
          ( ( x[3] == 0 ) AND ( k(3,0,3,0) OR k(3,3,4,0) )  ) OR
          
          ( ( x[0] == 1 ) AND K(0,0,1) ) OR 
          ( ( x[1] == 1 ) AND K(1,0,1) ) OR 
          ( ( x[2] == 1 ) AND K(2,0,1) ) OR 
          ( ( x[3] == 1 ) AND K(3,0,1) ) 
     ) 

l[0][2]:
     ( 
          ( l[0][1] ) OR 
          ( ( x[0] == 2 ) AND K(0,0,2) ) OR 
          ( ( x[1] == 2 ) AND K(1,0,2) ) OR 
          ( ( x[2] == 2 ) AND K(2,0,2) ) OR 
          ( ( x[3] == 2 ) AND K(3,0,2) ) 
     ) 

l[0][3]:
     ( 
          ( l[0][2] ) OR 
          ( ( x[0] == 3 ) AND K(0,0,3) ) OR 
          ( ( x[1] == 3 ) AND K(1,0,3) ) OR 
          ( ( x[2] == 3 ) AND K(2,0,3) ) OR 
          ( ( x[3] == 3 ) AND K(3,0,3) ) 
     )
          ( (x[0] == 1) AND 
               ( (x[0] == 0) AND (K(0,0,1,[0]) OR K(0,0,0,[])) ) OR
               ( (x[1] == 0) AND (K(1,0,0,[]) OR K(0,0,1,[1])) ) OR
               ( (x[2] == 0) AND (K(2,0,0,[]) OR K(0,0,1,[2])) ) OR
               ( (x[3] == 0) AND (K(3,0,0,[]) OR K(0,0,1,[3])) )
          
          OR
          
          ( (x[1] == 1) AND (x[0] == 0) AND (K(0,0,0,[]) OR K(1,0,1,[0])) ) OR
          ( (x[1] == 1) AND (x[1] == 0) AND (K(1,0,0,[]) OR K(1,0,1,[1])) ) OR
          ( (x[1] == 1) AND (x[2] == 0) AND (K(2,0,0,[]) OR K(1,0,1,[2])) ) OR
          ( (x[1] == 1) AND (x[3] == 0) AND (K(3,0,0,[]) OR K(1,0,1,[3])) )
          
          OR



          # ( x[1] == 1 ) AND ( x[1] == 0 ) ... OR # ---------- WRONG ----------
     ) 

l[0][1]: # Rewrite 4
     ( 
          ( (x[1] == 1) AND (x[0] == 0) AND (K(1,0,1,[0])) ) OR
          ( (x[1] == 1) AND (x[1] == 0) AND (K(1,0,1,[1])) ) OR
          ( (x[1] == 1) AND (x[2] == 0) AND (K(1,0,1,[2])) ) OR
          ( (x[1] == 1) AND (x[3] == 0) AND (K(1,0,1,[3])) )

          # ( x[1] == 1 ) AND ( x[1] == 0 ) ... OR # ---------- WRONG ----------
     ) 











l[0][1]:
     ( 
          ( ( x[0] == 0 ) AND 0 ) OR 
          ( ( x[1] == 0 ) AND k(1,0,1,0) ) OR 
          ( ( x[2] == 0 ) AND ( k(2,0,3,0) OR k(2,3,4,0) ) ) OR 
          ( ( x[3] == 0 ) AND ( k(3,0,3,0) OR k(3,3,4,0) )  ) OR
          
          ( ( x[0] == 1 ) AND K(0,0,1) ) OR 
          ( ( x[1] == 1 ) AND K(1,0,1) ) OR 
          ( ( x[2] == 1 ) AND K(2,0,1) ) OR 
          ( ( x[3] == 1 ) AND K(3,0,1) ) 
     ) 

l[0][2]:
     ( 
          ( l[0][1] ) OR 
          ( ( x[0] == 2 ) AND K(0,0,2) ) OR 
          ( ( x[1] == 2 ) AND K(1,0,2) ) OR 
          ( ( x[2] == 2 ) AND K(2,0,2) ) OR 
          ( ( x[3] == 2 ) AND K(3,0,2) ) 
     ) 

l[0][3]:
     ( 
          ( l[0][2] ) OR 
          ( ( x[0] == 3 ) AND K(0,0,3) ) OR 
          ( ( x[1] == 3 ) AND K(1,0,3) ) OR 
          ( ( x[2] == 3 ) AND K(2,0,3) ) OR 
          ( ( x[3] == 3 ) AND K(3,0,3) ) 
     )
          ( (x[0] == 1) AND 
               ( (x[0] == 0) AND (K(0,0,1,[0]) OR K(0,0,0,[])) ) OR
               ( (x[1] == 0) AND (K(1,0,0,[]) OR K(0,0,1,[1])) ) OR
               ( (x[2] == 0) AND (K(2,0,0,[]) OR K(0,0,1,[2])) ) OR
               ( (x[3] == 0) AND (K(3,0,0,[]) OR K(0,0,1,[3])) )
          
          OR
          
          ( (x[1] == 1) AND (x[0] == 0) AND (K(0,0,0,[]) OR K(1,0,1,[0])) ) OR
          ( (x[1] == 1) AND (x[1] == 0) AND (K(1,0,0,[]) OR K(1,0,1,[1])) ) OR
          ( (x[1] == 1) AND (x[2] == 0) AND (K(2,0,0,[]) OR K(1,0,1,[2])) ) OR
          ( (x[1] == 1) AND (x[3] == 0) AND (K(3,0,0,[]) OR K(1,0,1,[3])) )
          
          OR



          # ( x[1] == 1 ) AND ( x[1] == 0 ) ... OR # ---------- WRONG ----------
     ) 

l[0][1]: # Rewrite 4
     ( 
          ( (x[1] == 1) AND (x[0] == 0) AND (K(1,0,1,[0])) ) OR
          ( (x[1] == 1) AND (x[1] == 0) AND (K(1,0,1,[1])) ) OR
          ( (x[1] == 1) AND (x[2] == 0) AND (K(1,0,1,[2])) ) OR
          ( (x[1] == 1) AND (x[3] == 0) AND (K(1,0,1,[3])) )

          # ( x[1] == 1 ) AND ( x[1] == 0 ) ... OR # ---------- WRONG ----------
     ) 











l[0][1]:
     ( 
          ( ( x[0] == 0 ) AND 0 ) OR 
          ( ( x[1] == 0 ) AND k(1,0,1,0) ) OR 
          ( ( x[2] == 0 ) AND ( k(2,0,3,0) OR k(2,3,4,0) ) ) OR 
          ( ( x[3] == 0 ) AND ( k(3,0,3,0) OR k(3,3,4,0) )  ) OR
          
          ( ( x[0] == 1 ) AND K(0,0,1) ) OR 
          ( ( x[1] == 1 ) AND K(1,0,1) ) OR 
          ( ( x[2] == 1 ) AND K(2,0,1) ) OR 
          ( ( x[3] == 1 ) AND K(3,0,1) ) 
     ) 

l[0][2]:
     ( 
          ( l[0][1] ) OR 
          ( ( x[0] == 2 ) AND K(0,0,2) ) OR 
          ( ( x[1] == 2 ) AND K(1,0,2) ) OR 
          ( ( x[2] == 2 ) AND K(2,0,2) ) OR 
          ( ( x[3] == 2 ) AND K(3,0,2) ) 
     ) 

l[0][3]:
     ( 
          ( l[0][2] ) OR 
          ( ( x[0] == 3 ) AND K(0,0,3) ) OR 
          ( ( x[1] == 3 ) AND K(1,0,3) ) OR 
          ( ( x[2] == 3 ) AND K(2,0,3) ) OR 
          ( ( x[3] == 3 ) AND K(3,0,3) ) 
     )
          ( (x[0] == 1) AND 
               ( (x[0] == 0) AND (K(0,0,1,[0]) OR K(0,0,0,[])) ) OR
               ( (x[1] == 0) AND (K(1,0,0,[]) OR K(0,0,1,[1])) ) OR
               ( (x[2] == 0) AND (K(2,0,0,[]) OR K(0,0,1,[2])) ) OR
               ( (x[3] == 0) AND (K(3,0,0,[]) OR K(0,0,1,[3])) )
          
          OR
          
          ( (x[1] == 1) AND (x[0] == 0) AND (K(0,0,0,[]) OR K(1,0,1,[0])) ) OR
          ( (x[1] == 1) AND (x[1] == 0) AND (K(1,0,0,[]) OR K(1,0,1,[1])) ) OR
          ( (x[1] == 1) AND (x[2] == 0) AND (K(2,0,0,[]) OR K(1,0,1,[2])) ) OR
          ( (x[1] == 1) AND (x[3] == 0) AND (K(3,0,0,[]) OR K(1,0,1,[3])) )
          
          OR



          # ( x[1] == 1 ) AND ( x[1] == 0 ) ... OR # ---------- WRONG ----------
     ) 

l[0][1]: # Rewrite 4
     ( 
          ( (x[1] == 1) AND (x[0] == 0) AND (K(1,0,1,[0])) ) OR
          ( (x[1] == 1) AND (x[1] == 0) AND (K(1,0,1,[1])) ) OR
          ( (x[1] == 1) AND (x[2] == 0) AND (K(1,0,1,[2])) ) OR
          ( (x[1] == 1) AND (x[3] == 0) AND (K(1,0,1,[3])) )

          # ( x[1] == 1 ) AND ( x[1] == 0 ) ... OR # ---------- WRONG ----------
     ) 











l[0][1]:
     ( 
          ( ( x[0] == 0 ) AND 0 ) OR 
          ( ( x[1] == 0 ) AND k(1,0,1,0) ) OR 
          ( ( x[2] == 0 ) AND ( k(2,0,3,0) OR k(2,3,4,0) ) ) OR 
          ( ( x[3] == 0 ) AND ( k(3,0,3,0) OR k(3,3,4,0) )  ) OR
          
          ( ( x[0] == 1 ) AND K(0,0,1) ) OR 
          ( ( x[1] == 1 ) AND K(1,0,1) ) OR 
          ( ( x[2] == 1 ) AND K(2,0,1) ) OR 
          ( ( x[3] == 1 ) AND K(3,0,1) ) 
     ) 

l[0][2]:
     ( 
          ( l[0][1] ) OR 
          ( ( x[0] == 2 ) AND K(0,0,2) ) OR 
          ( ( x[1] == 2 ) AND K(1,0,2) ) OR 
          ( ( x[2] == 2 ) AND K(2,0,2) ) OR 
          ( ( x[3] == 2 ) AND K(3,0,2) ) 
     ) 

l[0][3]:
     ( 
          ( l[0][2] ) OR 
          ( ( x[0] == 3 ) AND K(0,0,3) ) OR 
          ( ( x[1] == 3 ) AND K(1,0,3) ) OR 
          ( ( x[2] == 3 ) AND K(2,0,3) ) OR 
          ( ( x[3] == 3 ) AND K(3,0,3) ) 
     )
          ( (x[0] == 1) AND 
               ( (x[0] == 0) AND (K(0,0,1,[0]) OR K(0,0,0,[])) ) OR
               ( (x[1] == 0) AND (K(1,0,0,[]) OR K(0,0,1,[1])) ) OR
               ( (x[2] == 0) AND (K(2,0,0,[]) OR K(0,0,1,[2])) ) OR
               ( (x[3] == 0) AND (K(3,0,0,[]) OR K(0,0,1,[3])) )
          
          OR
          
          ( (x[1] == 1) AND (x[0] == 0) AND (K(0,0,0,[]) OR K(1,0,1,[0])) ) OR
          ( (x[1] == 1) AND (x[1] == 0) AND (K(1,0,0,[]) OR K(1,0,1,[1])) ) OR
          ( (x[1] == 1) AND (x[2] == 0) AND (K(2,0,0,[]) OR K(1,0,1,[2])) ) OR
          ( (x[1] == 1) AND (x[3] == 0) AND (K(3,0,0,[]) OR K(1,0,1,[3])) )
          
          OR



          # ( x[1] == 1 ) AND ( x[1] == 0 ) ... OR # ---------- WRONG ----------
     ) 

l[0][1]: # Rewrite 4
     ( 
          ( (x[1] == 1) AND (x[0] == 0) AND (K(1,0,1,[0])) ) OR
          ( (x[1] == 1) AND (x[1] == 0) AND (K(1,0,1,[1])) ) OR
          ( (x[1] == 1) AND (x[2] == 0) AND (K(1,0,1,[2])) ) OR
          ( (x[1] == 1) AND (x[3] == 0) AND (K(1,0,1,[3])) )

          # ( x[1] == 1 ) AND ( x[1] == 0 ) ... OR # ---------- WRONG ----------
     ) 











l[0][1]:
     ( 
          ( ( x[0] == 0 ) AND 0 ) OR 
          ( ( x[1] == 0 ) AND k(1,0,1,0) ) OR 
          ( ( x[2] == 0 ) AND ( k(2,0,3,0) OR k(2,3,4,0) ) ) OR 
          ( ( x[3] == 0 ) AND ( k(3,0,3,0) OR k(3,3,4,0) )  ) OR
          
          ( ( x[0] == 1 ) AND K(0,0,1) ) OR 
          ( ( x[1] == 1 ) AND K(1,0,1) ) OR 
          ( ( x[2] == 1 ) AND K(2,0,1) ) OR 
          ( ( x[3] == 1 ) AND K(3,0,1) ) 
     ) 

l[0][2]:
     ( 
          ( l[0][1] ) OR 
          ( ( x[0] == 2 ) AND K(0,0,2) ) OR 
          ( ( x[1] == 2 ) AND K(1,0,2) ) OR 
          ( ( x[2] == 2 ) AND K(2,0,2) ) OR 
          ( ( x[3] == 2 ) AND K(3,0,2) ) 
     ) 

l[0][3]:
     ( 
          ( l[0][2] ) OR 
          ( ( x[0] == 3 ) AND K(0,0,3) ) OR 
          ( ( x[1] == 3 ) AND K(1,0,3) ) OR 
          ( ( x[2] == 3 ) AND K(2,0,3) ) OR 
          ( ( x[3] == 3 ) AND K(3,0,3) ) 
     )
          ( (x[0] == 1) AND 
               ( (x[0] == 0) AND (K(0,0,1,[0]) OR K(0,0,0,[])) ) OR
               ( (x[1] == 0) AND (K(1,0,0,[]) OR K(0,0,1,[1])) ) OR
               ( (x[2] == 0) AND (K(2,0,0,[]) OR K(0,0,1,[2])) ) OR
               ( (x[3] == 0) AND (K(3,0,0,[]) OR K(0,0,1,[3])) )
          
          OR
          
          ( (x[1] == 1) AND (x[0] == 0) AND (K(0,0,0,[]) OR K(1,0,1,[0])) ) OR
          ( (x[1] == 1) AND (x[1] == 0) AND (K(1,0,0,[]) OR K(1,0,1,[1])) ) OR
          ( (x[1] == 1) AND (x[2] == 0) AND (K(2,0,0,[]) OR K(1,0,1,[2])) ) OR
          ( (x[1] == 1) AND (x[3] == 0) AND (K(3,0,0,[]) OR K(1,0,1,[3])) )
          
          OR



          # ( x[1] == 1 ) AND ( x[1] == 0 ) ... OR # ---------- WRONG ----------
     ) 

l[0][1]: # Rewrite 4
     ( 
          ( (x[1] == 1) AND (x[0] == 0) AND (K(1,0,1,[0])) ) OR
          ( (x[1] == 1) AND (x[1] == 0) AND (K(1,0,1,[1])) ) OR
          ( (x[1] == 1) AND (x[2] == 0) AND (K(1,0,1,[2])) ) OR
          ( (x[1] == 1) AND (x[3] == 0) AND (K(1,0,1,[3])) )

          # ( x[1] == 1 ) AND ( x[1] == 0 ) ... OR # ---------- WRONG ----------
     ) 











l[0][1]:
     ( 
          ( ( x[0] == 0 ) AND 0 ) OR 
          ( ( x[1] == 0 ) AND k(1,0,1,0) ) OR 
          ( ( x[2] == 0 ) AND ( k(2,0,3,0) OR k(2,3,4,0) ) ) OR 
          ( ( x[3] == 0 ) AND ( k(3,0,3,0) OR k(3,3,4,0) )  ) OR
          
          ( ( x[0] == 1 ) AND K(0,0,1) ) OR 
          ( ( x[1] == 1 ) AND K(1,0,1) ) OR 
          ( ( x[2] == 1 ) AND K(2,0,1) ) OR 
          ( ( x[3] == 1 ) AND K(3,0,1) ) 
     ) 

l[0][2]:
     ( 
          ( l[0][1] ) OR 
          ( ( x[0] == 2 ) AND K(0,0,2) ) OR 
          ( ( x[1] == 2 ) AND K(1,0,2) ) OR 
          ( ( x[2] == 2 ) AND K(2,0,2) ) OR 
          ( ( x[3] == 2 ) AND K(3,0,2) ) 
     ) 

l[0][3]:
     ( 
          ( l[0][2] ) OR 
          ( ( x[0] == 3 ) AND K(0,0,3) ) OR 
          ( ( x[1] == 3 ) AND K(1,0,3) ) OR 
          ( ( x[2] == 3 ) AND K(2,0,3) ) OR 
          ( ( x[3] == 3 ) AND K(3,0,3) ) 
     )

SyntaxError: invalid syntax (1235548119.py, line 78)

In [ ]:
vector_k = k(2,2,4,1)
vector_k = clean_vector(vector_k)

print_vector(vector_k)

k(2,2,4,1) 
